In [1]:
import os

In [2]:
%pwd

'd:\\Code\\Git\\myprojects\\Equation-Solver-End-to-End\\Handwritten_Equation_Solver\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd


'd:\\Code\\Git\\myprojects\\Equation-Solver-End-to-End\\Handwritten_Equation_Solver'

In [5]:
from dataclasses import dataclass
from pathlib import Path
from equation_solver.utils.common import get_unique_class_names

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    classes_ideal: list

In [6]:
from equation_solver.constants import *
from equation_solver.utils.common import read_yaml, create_directories


In [7]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf

[2024-11-06 12:42:10,503: WARNING: module_wrapper: From d:\Code\Git\myprojects\Equation-Solver-End-to-End\Handwritten_Equation_Solver\.eq_solver_venv\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "symbol_data")
        classes_ideal = get_unique_class_names(training_data)
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            classes_ideal=classes_ideal
        )

        return training_config

In [9]:
class DataGenerator:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.class_indices = {name: idx for idx, name in enumerate(self.config.classes_ideal)}

    def load_data(self):
        all_files = []
        for root, dirs, files in os.walk(self.config.training_data):
            for file in files:
                if file.endswith(('.png', '.jpg', '.jpeg')):
                    class_name = os.path.basename(root)
                    if class_name in self.class_indices:
                        file_path = os.path.join(root, file)
                        all_files.append((file_path, class_name))

        # print(f"Total files found: {len(all_files)}")
        # print(f"Number of classes: {len(self.class_indices)}")
        # print(f"Class indices: {self.class_indices}")

        # Shuffle the files
        np.random.shuffle(all_files)

        # Split file paths and labels
        file_paths, class_names = zip(*all_files)
        labels = [self.class_indices[class_name] for class_name in class_names]

        # Perform stratified split
        file_paths_train, file_paths_val, labels_train, labels_val = train_test_split(
            file_paths, labels, test_size=0.2, stratify=labels, random_state=34
        )

        # print(f"Training samples: {len(file_paths_train)}")
        # print(f"Validation samples: {len(file_paths_val)}")

        # Oversample the training data
        oversampler = RandomOverSampler(sampling_strategy='auto', random_state=54)
        file_paths_train_resampled, labels_train_resampled = oversampler.fit_resample(
            np.array(file_paths_train).reshape(-1, 1), labels_train
        )
        file_paths_train_resampled = file_paths_train_resampled.flatten()

        # Create datasets
        train_ds = tf.data.Dataset.from_tensor_slices((file_paths_train_resampled, labels_train_resampled))
        val_ds = tf.data.Dataset.from_tensor_slices((file_paths_val, labels_val))

        # Apply preprocessing
        train_ds = train_ds.map(self.load_image, num_parallel_calls=tf.data.AUTOTUNE)
        val_ds = val_ds.map(self.load_image, num_parallel_calls=tf.data.AUTOTUNE)

        # Shuffle and batch
        train_ds = train_ds.cache().shuffle(buffer_size=len(file_paths_train_resampled)).batch(self.config.params_batch_size).prefetch(tf.data.AUTOTUNE)
        val_ds = val_ds.cache().batch(self.config.params_batch_size).prefetch(tf.data.AUTOTUNE)

        return train_ds, val_ds

    def load_image(self, file_path, label):
        image = tf.io.read_file(file_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, self.config.params_image_size[:-1])
        image = tf.image.rgb_to_grayscale(image)
        return image, label

In [10]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.batch_size = self.config.params_batch_size
        self.data_generator = DataGenerator(config)
        self.train_ds, self.val_ds = self.data_generator.load_data()
        # print(f"Using batch size: {self.batch_size}")

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
        input_shape = self.model.input_shape
        # print(f"Model expects input shape: {input_shape}")
        # self.model.summary()

    def train_valid_generator(self):
        pass

    def train(self):
        # Calculate steps properly
        self.steps_per_epoch = len(self.train_ds)
        self.validation_steps = len(self.val_ds)

        self.model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        # Train the model
        history = self.model.fit(
            self.train_ds,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.val_ds,
            verbose=1
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
        
        return history

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [12]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e


[2024-11-06 12:44:06,096: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-06 12:44:06,099: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-06 12:44:06,101: INFO: common: created directory at: artifacts]
[2024-11-06 12:44:06,115: INFO: common: created directory at: artifacts\training]
[2024-11-06 12:44:06,521: WARNING: module_wrapper: From d:\Code\Git\myprojects\Equation-Solver-End-to-End\Handwritten_Equation_Solver\.eq_solver_venv\Lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.
]
[2024-11-06 12:44:06,917: WARNING: module_wrapper: From d:\Code\Git\myprojects\Equation-Solver-End-to-End\Handwritten_Equation_Solver\.eq_solver_venv\Lib\site-packages\keras\src\optimizers\__init__.py:309: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.
]
Epoch 1/5
[2024-11-06 12:44:07,271: WARNI

d:\Code\Git\myprojects\Equation-Solver-End-to-End\Handwritten_Equation_Solver\.eq_solver_venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
